In [2]:
import pandas as pd
import numpy as np


In [4]:
#sumber data untuk di cleansing
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSwyygYNEimDNePuip_cs10fB-qP8H2tKZuBnTnVRmhn0-Dpt6K3-5qfszGVcg9R0a27EWhBIjJOweg/pub?output=csv"
df = pd.read_csv(url)


In [6]:
#pengecekan data awal
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID Pelanggan   105 non-null    object
 1   Nama Klinik    105 non-null    object
 2   Nama Sales     105 non-null    object
 3   Bulan Join     105 non-null    object
 4   Jumlah Klinik  105 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 4.2+ KB


In [8]:
#mengstandarisasi kolom data
df.columns = (
    df.columns
      .str.lower()
      .str.strip()
      .str.replace(" ", "_")
)


In [9]:
#menghilangan simbol dan kelebihan spaci
text_cols = ['nama_klinik', 'nama_sales', 'bulan_join']

for col in text_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.title()
    )


In [12]:
#data map untuk bulan karna inkonsisten penamaan perseperti (ind , en , dan typo)
bulan_map = {
    "Januari": 1, "January": 1,
    "Februari": 2, "February": 2,
    "Maret": 3, "March": 3,
    "April": 4,
    "Mei": 5, "May": 5,
    "Juni": 6, "June": 6,
    "Juli": 7, "July": 7,
    "Agustus": 8, "Agustis": 8, "August": 8,
    "September": 9,
    "Oktober": 10, "October": 10,
    "November": 11,
    "Desember": 12, "December": 12
}
df['bulan_join_num'] = df['bulan_join'].map(bulan_map)

In [14]:
#validasi hasil mapping data (jika kosong maka mapping tersebut aman)
df[df['bulan_join_num'].isna()][['bulan_join']]


,bulan_join


In [16]:
#validasi tipe data
df['jumlah_klinik'] = pd.to_numeric(df['jumlah_klinik'], errors='coerce')
df[df['jumlah_klinik'].isna()]


,id_pelanggan,nama_klinik,nama_sales,bulan_join,jumlah_klinik,bulan_join_num


In [19]:
#pengecekan data duplikat
df.duplicated().sum()
df[df.duplicated(keep=False)].sort_values('id_pelanggan')


,id_pelanggan,nama_klinik,nama_sales,bulan_join,jumlah_klinik,bulan_join_num
15,PEL00013205,Klinik Peduli 04,Sales E,Desember,2,12
50,PEL00013205,Klinik Peduli 04,Sales E,Desember,2,12
21,PEL00013603,Klinik Asri 01,Sales E,Maret,3,3
40,PEL00013603,Klinik Asri 01,Sales E,Maret,3,3
61,PEL00023671,Klinik Asri 07,Sales B,November,2,11
92,PEL00023671,Klinik Asri 07,Sales B,November,2,11
52,PEL00053205,Klinik Peduli 05,Sales B,Desember,2,12
78,PEL00053205,Klinik Peduli 05,Sales B,Desember,2,12
44,PEL00353671,Klinik Prima 07,Sales E,April,1,4
65,PEL00353671,Klinik Prima 07,Sales E,April,1,4


In [24]:
#dari data di atas terdapat beberapa data duplikat yg harus di bersihkan ada 2 cara yg pertama di hapus yg kedua pengunakan flagging sebagai tanda
#kita pilih opsi kedua untuk mengatasi data duplikat tersebut dengan mencatumkan (is_duplicate)
df_clean = df.drop_duplicates()
df['is_duplicate'] = df.duplicated()



In [26]:
#menyusun final kolom untuk data yg sudah di bersihkan
final_cols = [
    'id_pelanggan',
    'nama_klinik',
    'nama_sales',
    'bulan_join',
    'bulan_join_num',
    'jumlah_klinik'
]

df_final = df_clean[final_cols]


In [27]:
#cek hasil data yg sudah di cleansing
df_final.sample(5)
df_final.describe()


,bulan_join_num,jumlah_klinik
count,100.000000,100.000000
mean,6.830000,2.520000
std,3.499076,1.201682
min,1.000000,1.000000
25%,4.000000,1.000000
50%,7.000000,2.500000
75%,10.000000,3.000000
max,12.000000,5.000000


In [29]:
#merubah data menjadi file
df_final.to_csv("data_klinik_clean.csv", index=False)


In [30]:
from google.colab import files
files.download("data_klinik_clean.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>